<a href="https://colab.research.google.com/github/RohaidAhmed/DR-Train-Dataset/blob/main/clahe_training_densenet_epoch32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.applications import InceptionResNetV2
from keras.layers.core import Lambda
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.inception_v3 import InceptionV3 
from keras.applications.densenet import DenseNet121
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import *
# from keras.utils.multi_gpu_utils import multi_gpu_model
from matplotlib import pyplot as plt
from keras.models import load_model
import keras.backend as K
import os
import tensorflow as tf


import numpy as np
import pandas as pd
from PIL import Image
import cv2 

import sys
import os.path

In [ ]:
print(tf.config.list_physical_devices('GPU'))

[]


In [ ]:

def Global_attention_block(inputs):
    shape=K.int_shape(inputs)
    x=AveragePooling2D(pool_size=(shape[1],shape[2])) (inputs)
    x=Conv2D(shape[3],1, padding='same') (x)
    x=Activation('relu') (x)
    x=Conv2D(shape[3],1, padding='same') (x)
    x=Activation('sigmoid') (x)
    C_A=Multiply()([x,inputs])
    
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))  (C_A)
    x=Activation('sigmoid') (x)
    S_A=Multiply()([x,C_A])
    return S_A

In [ ]:
def Category_attention_block(inputs,classes,k):
    shape=K.int_shape(inputs)
    F=Conv2D(k*classes,1, padding='same') (inputs)
    F=BatchNormalization() (F)
    F1=Activation('relu') (F)
    
    F2=F1
    x=GlobalMaxPool2D()(F2)
    
    x=Reshape((classes,k)) (x)
    S=Lambda(lambda x: K.mean(x,axis=-1,keepdims=False))  (x)
    
    x=Reshape((shape[1],shape[2],classes,k)) (F1)
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=False))  (x)
    x=Multiply()([S,x])
    M=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))  (x)
    
    semantic=Multiply()([inputs,M])
    return semantic


In [ ]:
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points    
   

In [ ]:
def plotmodel(history,name):
    
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1) 
    
    plt.figure(1)                  
    plt.plot(epochs,smooth_curve(acc))
    plt.plot(epochs,smooth_curve(val_acc))
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.legend(['train_acc', 'val_acc'], loc='upper left')
    plt.savefig('acc_'+name+'.png')
    
    plt.figure(2)
    plt.plot(epochs,smooth_curve(loss))
    plt.plot(epochs,smooth_curve(val_loss))
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'val_loss'], loc='upper right')
    plt.savefig('loss_'+name+'.png')

In [ ]:
def label_smooth(y_true, y_pred):
    y_true=((1-0.1)*y_true+0.05)
    return K.categorical_crossentropy(y_true, y_pred) 
 

In [ ]:
def get_base_model(model_name,image_size):
    if model_name =='vgg16':
        base_model=VGG16              (include_top=False,weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='resnet50':
        base_model=ResNet50           (include_top=False,weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='xception':
        base_model=Xception           (include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='densenet121':
        base_model=DenseNet121       (include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='mobilenet0.75':
        base_model=MobileNet         (include_top=False,weights='imagenet',alpha=0.75,input_shape=(image_size,image_size,3))
    if model_name =='mobilenet1.0':
        base_model=MobileNet         (include_top=False,weights='imagenet',alpha=1.0,input_shape=(image_size,image_size,3))
    if model_name =='mobilenetv2':
        base_model=MobileNetV2      (include_top=False,weights='imagenet',alpha=1.0,input_shape=(image_size,image_size,3))
    if model_name =='inceptionv3':   
        base_model=InceptionV3       (include_top=False,weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='inceptionv2':
        base_model=InceptionResNetV2 (include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    return base_model

In [ ]:
# def clahe(img):
# #     for img in os.listdir(path):
# #         imgarray = Image.open(os.path.join(path, img))
# #     img = cv2.imread(img)
#     lab = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
    

#         # -----Splitting the LAB image to different channels-------------------------
#     l, a, b = cv2.split(lab)

#         # -----Applying CLAHE to L-channel-------------------------------------------
#     clahe = cv2.createCLAHE(clipLimit=0.02, tileGridSize=(5,5))
#     lab[:,:,0] = clahe.apply(lab[:,:,0])
# #     cl = clahe.apply(lab)

#         # -----Merge the CLAHE enhanced L-channel with the a and b channel-----------
# #     limg = cv2.merge((cl, a, b))

#         # -----Converting image from LAB Color model to RGB model--------------------
#     final = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
#     return final
# def clahe(img):
#     lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
#     l, a, b = cv2.split(lab)
#     clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
#     cl = clahe.apply(l)
#     limg = cv2.merge((cl,a,b))
#     final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
#     return final

In [ ]:
def clahe(image):
    if image.dtype == np.uint16:
        image = (image / 256).astype(np.uint8)  # Convert 16-bit image to 8-bit
    elif image.dtype != np.uint8:
        image = image.astype(np.uint8)  # Convert other data types to 8-bit
    image = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    clahe = cv2.createCLAHE(clipLimit=0.02, tileGridSize=(5,5))
    image[:,:,0] = clahe.apply(image[:,:,0])
    image = cv2.cvtColor(image, cv2.COLOR_LAB2RGB)
    return image

In [ ]:
def train_model(model,dataset,image_size,batch_size,save_name,lr1,lr2,Epochs1,Epochs2):
    
    #dataParam={'messidor': [957,243,2,'./data/messidor/train','./data/messidor/test'],
              #  'kaggle': [30000,5126,5,'./data/kaggle/train','./data/kaggle/valid'],
              #  'DDR':   [9851,2503,5,'./data/DDR/train','./data/DDR/valid']} #6119
    dataParam = {'DR': [5597,1868,4,r'D:\FYP_Retinopathy\Rohaid\CABNET\Densenet121\CABnet-master\CABnet-master\data\train',
                        r'D:\FYP_Retinopathy\Rohaid\CABNET\Densenet121\CABnet-master\CABnet-master\data\test'],
                'DDR':   [10015,2507,5,r'C:\Users\FYP_Retinopathy\Documents\CABNET\DDR\Dataset\train',
                          r'C:\Users\FYP_Retinopathy\Documents\CABNET\DDR\Dataset\test'],
                'DRgrade': [5003,1253,4,r'C:\Users\FYP_Retinopathy\Documents\CABNET\DDR\Network3\Dataset\train',
                            r'C:\Users\FYP_Retinopathy\Documents\CABNET\DDR\Network3\Dataset\test']}
#     trainC=[]
#     testC=[]
    train_num,valid_num,classes,train_dir,test_dir = dataParam[dataset]
    
#     trainC = clahe(cliplimit,gridsize,train_dir)
#     testC = clahe(cliplimit,gridsize,test_dir)
    
    
    train=ImageDataGenerator(preprocessing_function=clahe,horizontal_flip=True,vertical_flip=True,rotation_range=90)          
    valid = ImageDataGenerator(preprocessing_function= clahe,horizontal_flip=True,vertical_flip=True,rotation_range=90)
    train_data=train.flow_from_directory(train_dir,
                                         target_size=(image_size,image_size),
                                         shuffle = True,
                                         batch_size=batch_size)
    valid_data=valid.flow_from_directory(test_dir,
                                         target_size=(image_size,image_size),
                                         shuffle = False,
                                         batch_size=batch_size)

    lr_decay=ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, verbose=1)
    save_model=ModelCheckpoint('new/'+save_name+'{epoch:02d}.h5', monitor='val_loss',save_freq=10)
    
    for layer in base_model.layers:
        layer.trainable = False   
        
    model.compile(optimizer=Adam(learning_rate=lr1,decay=0.00001),loss=loss_fun,metrics=['acc'])
    model.fit(train_data,
                        steps_per_epoch=train_num/batch_size,
                        validation_data=valid_data,
                        validation_steps=valid_num/batch_size,
                        epochs=Epochs1, 
                        workers=2,
                        callbacks=[lr_decay,save_model])   
    
    for layer in base_model.layers:
        layer.trainable = True
        
    model.compile(optimizer=Adam(learning_rate=lr2,decay=0.00001),loss=loss_fun,metrics=['acc'])
    history=model.fit(train_data,
                        steps_per_epoch=train_num/batch_size,
                        validation_data=valid_data,
                        validation_steps=valid_num/batch_size,
                        epochs=Epochs2,
                        workers=2,
                        callbacks=[lr_decay,save_model])
    return history


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"    
loss_fun= 'categorical_crossentropy'  
gpu_num=1
k=5
lr1=0.005
lr2=0.0001
batch_size= 16
image_size=512
classes=4

base_model=get_base_model('densenet121',image_size)  
base_in=base_model.input
base_out=base_model.output

x=Global_attention_block(base_out)
base_out=Category_attention_block(x,classes,k)

x=GlobalAveragePooling2D()(base_out)
out=Dense(classes,activation='softmax')(x)

if gpu_num>1:
    model=Model(base_model.input,out)
    #model.summary()
    parallel_model = multi_gpu_model(model, gpus=gpu_num)
    parallel_model.summary()
else:
    parallel_model=Model(base_model.input,out)
    parallel_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 518, 518, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 256, 256, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

 conv2_block3_concat (Concatena  (None, 128, 128, 16  0          ['conv2_block2_concat[0][0]',    
 te)                            0)                                'conv2_block3_2_conv[0][0]']    
                                                                                                  
 conv2_block4_0_bn (BatchNormal  (None, 128, 128, 16  640        ['conv2_block3_concat[0][0]']    
 ization)                       0)                                                                
                                                                                                  
 conv2_block4_0_relu (Activatio  (None, 128, 128, 16  0          ['conv2_block4_0_bn[0][0]']      
 n)                             0)                                                                
                                                                                                  
 conv2_block4_1_conv (Conv2D)   (None, 128, 128, 12  20480       ['conv2_block4_0_relu[0][0]']    
          

 conv3_block1_1_conv (Conv2D)   (None, 64, 64, 128)  16384       ['conv3_block1_0_relu[0][0]']    
                                                                                                  
 conv3_block1_1_bn (BatchNormal  (None, 64, 64, 128)  512        ['conv3_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block1_1_relu (Activatio  (None, 64, 64, 128)  0          ['conv3_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block1_2_conv (Conv2D)   (None, 64, 64, 32)   36864       ['conv3_block1_1_relu[0][0]']    
                                                                                                  
 conv3_blo

                                                                                                  
 conv3_block5_2_conv (Conv2D)   (None, 64, 64, 32)   36864       ['conv3_block5_1_relu[0][0]']    
                                                                                                  
 conv3_block5_concat (Concatena  (None, 64, 64, 288)  0          ['conv3_block4_concat[0][0]',    
 te)                                                              'conv3_block5_2_conv[0][0]']    
                                                                                                  
 conv3_block6_0_bn (BatchNormal  (None, 64, 64, 288)  1152       ['conv3_block5_concat[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block6_0_relu (Activatio  (None, 64, 64, 288)  0          ['conv3_block6_0_bn[0][0]']      
 n)       

 lization)                                                                                        
                                                                                                  
 conv3_block10_0_relu (Activati  (None, 64, 64, 416)  0          ['conv3_block10_0_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv3_block10_1_conv (Conv2D)  (None, 64, 64, 128)  53248       ['conv3_block10_0_relu[0][0]']   
                                                                                                  
 conv3_block10_1_bn (BatchNorma  (None, 64, 64, 128)  512        ['conv3_block10_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv3_blo

...

 conv4_block23_concat (Concaten  (None, 32, 32, 992)  0          ['conv4_block22_concat[0][0]',   
 ate)                                                             'conv4_block23_2_conv[0][0]']   
                                                                                                  
 conv4_block24_0_bn (BatchNorma  (None, 32, 32, 992)  3968       ['conv4_block23_concat[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block24_0_relu (Activati  (None, 32, 32, 992)  0          ['conv4_block24_0_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block24_1_conv (Conv2D)  (None, 32, 32, 128)  126976      ['conv4_block24_0_relu[0][0]']   
          

                                                                                                  
 conv5_block3_2_conv (Conv2D)   (None, 16, 16, 32)   36864       ['conv5_block3_1_relu[0][0]']    
                                                                                                  
 conv5_block3_concat (Concatena  (None, 16, 16, 608)  0          ['conv5_block2_concat[0][0]',    
 te)                                                              'conv5_block3_2_conv[0][0]']    
                                                                                                  
 conv5_block4_0_bn (BatchNormal  (None, 16, 16, 608)  2432       ['conv5_block3_concat[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block4_0_relu (Activatio  (None, 16, 16, 608)  0          ['conv5_block4_0_bn[0][0]']      
 n)       

 ization)                                                                                         
                                                                                                  
 conv5_block8_0_relu (Activatio  (None, 16, 16, 736)  0          ['conv5_block8_0_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block8_1_conv (Conv2D)   (None, 16, 16, 128)  94208       ['conv5_block8_0_relu[0][0]']    
                                                                                                  
 conv5_block8_1_bn (BatchNormal  (None, 16, 16, 128)  512        ['conv5_block8_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_blo

 conv5_block12_1_bn (BatchNorma  (None, 16, 16, 128)  512        ['conv5_block12_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv5_block12_1_relu (Activati  (None, 16, 16, 128)  0          ['conv5_block12_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv5_block12_2_conv (Conv2D)  (None, 16, 16, 32)   36864       ['conv5_block12_1_relu[0][0]']   
                                                                                                  
 conv5_block12_concat (Concaten  (None, 16, 16, 896)  0          ['conv5_block11_concat[0][0]',   
 ate)                                                             'conv5_block12_2_conv[0][0]']   
          

                                                                                                  
 conv5_block16_concat (Concaten  (None, 16, 16, 1024  0          ['conv5_block15_concat[0][0]',   
 ate)                           )                                 'conv5_block16_2_conv[0][0]']   
                                                                                                  
 bn (BatchNormalization)        (None, 16, 16, 1024  4096        ['conv5_block16_concat[0][0]']   
                                )                                                                 
                                                                                                  
 relu (Activation)              (None, 16, 16, 1024  0           ['bn[0][0]']                     
                                )                                                                 
                                                                                                  
 average_p

In [ ]:
# with tf.device('/GPU:0'):
history=train_model(parallel_model,'DRgrade',image_size,batch_size,'densenet121',lr1,lr2,1,70)



Found 5003 images belonging to 4 classes.
Found 1253 images belonging to 4 classes.
312/312 [==============================] - 2359s 7s/step - loss: 0.9404 - acc: 0.7144 - val_loss: 0.8862 - val_acc: 0.7151 - lr: 0.0050
Epoch 1/70
312/312 [==============================] - 8991s 29s/step - loss: 0.7367 - acc: 0.7438 - val_loss: 0.7578 - val_acc: 0.7837 - lr: 1.0000e-04
Epoch 2/70
312/312 [==============================] - 8597s 27s/step - loss: 0.6038 - acc: 0.7847 - val_loss: 0.6480 - val_acc: 0.7885 - lr: 1.0000e-04
Epoch 3/70
312/312 [==============================] - 8598s 27s/step - loss: 0.5212 - acc: 0.8047 - val_loss: 0.6507 - val_acc: 0.7438 - lr: 1.0000e-04
Epoch 4/70
312/312 [==============================] - 8596s 27s/step - loss: 0.4655 - acc: 0.8191 - val_loss: 0.4600 - val_acc: 0.8252 - lr: 1.0000e-04
Epoch 5/70
312/312 [==============================] - 8595s 27s/step - loss: 0.4233 - acc: 0.8315 - val_loss: 0.4323 - val_acc: 0.8300 - lr: 1.0000e-04
Epoch 6/70
312/312 [

Epoch 42/70
312/312 [==============================] - 8595s 27s/step - loss: 0.0287 - acc: 0.9912 - val_loss: 0.5308 - val_acc: 0.8587 - lr: 8.5899e-06
Epoch 43/70
312/312 [==============================] - 8559s 27s/step - loss: 0.0281 - acc: 0.9918 - val_loss: 0.5798 - val_acc: 0.8707 - lr: 8.5899e-06
Epoch 44/70
313/312 [==============================] - ETA: -8s - loss: 0.0276 - acc: 0.9916
Epoch 44: ReduceLROnPlateau reducing learning rate to 6.871946970932186e-06.
312/312 [==============================] - 8942s 29s/step - loss: 0.0276 - acc: 0.9916 - val_loss: 0.5821 - val_acc: 0.8579 - lr: 8.5899e-06
Epoch 45/70
312/312 [==============================] - 8571s 27s/step - loss: 0.0243 - acc: 0.9930 - val_loss: 0.5914 - val_acc: 0.8643 - lr: 6.8719e-06
Epoch 46/70
312/312 [==============================] - 8564s 27s/step - loss: 0.0216 - acc: 0.9936 - val_loss: 0.5569 - val_acc: 0.8563 - lr: 6.8719e-06
Epoch 47/70
313/312 [==============================] - ETA: -8s - loss: 0.020

In [ ]:
plotmodel(history,'densenet121')